In [83]:
from gen_t2m_c import *

Inital code
encode-> add motion in between -> decode
encode the prev motion(bvh) -> 

In [84]:
import sys
sys.argv = ['']  # Prevent argparse from picking up unwanted Jupyter arguments

# from gen_t2m import EvalT2MOptions  # or from wherever your file is

# Initialize and parse
parser = EvalT2MOptions()
opt = parser.parse(is_eval=True)

# Set values as per your CLI
opt.res_name = "tres_nlayer8_ld384_ff1024_rvq6ns_cdp0.2_sw"
opt.name = "2024-02-14-14-27-29_8_GPT_officialTrans_2iterPrdictEnd"
opt.text_prompt = "the person crouches and walks forward."
opt.motion_length = -1
opt.repeat_times = 3
opt.gpu_id = -1
opt.seed = 1
opt.ext = "generation_name_nopredlen"


------------ Options -------------
batch_size: 32
checkpoints_dir: ./log/t2m
cond_scale: 4
dataset_name: t2m
dropout: 0.2
ext: text2motion
ff_size: 1024
force_mask: False
gpu_id: -1
gumbel_sample: False
is_train: False
latent_dim: 384
mask_edit_section: None
max_motion_length: 196
motion_length: 0
n_heads: 6
n_layers: 8
name: t2m_nlayer8_nhead6_ld384_ff1024_cdp0.1_rvq6ns
num_batch: 2
repeat_times: 1
res_name: tres_nlayer8_ld384_ff1024_rvq6ns_cdp0.2_sw
seed: 1
source_motion: example_data/000612.npy
temperature: 1.0
text_path: 
text_prompt: 
time_steps: 10
topkr: 0.9
unit_length: 4
use_res_model: False
vq_name: rvq_nq1_dc512_nc512
which_epoch: latest
-------------- End ----------------


In [85]:
fixseed(opt.seed)

opt.device = torch.device("cpu" if opt.gpu_id == -1 else "cuda:" + str(opt.gpu_id))
torch.autograd.set_detect_anomaly(True)

dim_pose = 251 if opt.dataset_name == 'kit' else 263

# out_dir = pjoin(opt.check)
root_dir = pjoin(opt.checkpoints_dir, opt.dataset_name, opt.name)
model_dir = pjoin(root_dir, 'model')
result_dir = pjoin('./generation', opt.ext)
joints_dir = pjoin(result_dir, 'joints')
animation_dir = pjoin(result_dir, 'animations')
os.makedirs(joints_dir, exist_ok=True)
os.makedirs(animation_dir,exist_ok=True)

In [86]:
model_opt_path = pjoin(root_dir, 'opt.txt')
model_opt = get_opt(model_opt_path, device=opt.device)

Reading ./log/t2m/t2m/2024-02-14-14-27-29_8_GPT_officialTrans_2iterPrdictEnd/opt.txt


In [87]:
global vq_opt # This now works as vq_opt is global
# generator = BVHGenerator()

vq_opt_path = pjoin('./log/vq', opt.dataset_name, model_opt.vq_name, 'opt.txt')
vq_opt = get_opt(vq_opt_path, device=opt.device)
vq_model , vq_opt = load_vq_model(vq_opt)

Reading ./log/vq/t2m/rvq_nq6_dc512_nc512_noshare_qdp0.2/opt.txt
Loading VQ Model rvq_nq6_dc512_nc512_noshare_qdp0.2 Completed!


In [88]:
model_opt.num_tokens = vq_opt.nb_code
model_opt.num_quantizers = vq_opt.num_quantizers
model_opt.code_dim = vq_opt.code_dim

In [89]:
def load_res_model(res_opt,vq_opt):
    res_opt.num_quantizers = vq_opt.num_quantizers
    res_opt.num_tokens = vq_opt.nb_code
    res_transformer = ResidualTransformer(code_dim=vq_opt.code_dim,
                                            cond_mode='text',
                                            latent_dim=res_opt.latent_dim,
                                            ff_size=res_opt.ff_size,
                                            num_layers=res_opt.n_layers,
                                            num_heads=res_opt.n_heads,
                                            dropout=res_opt.dropout,
                                            clip_dim=512,
                                            shared_codebook=vq_opt.shared_codebook,
                                            cond_drop_prob=res_opt.cond_drop_prob,
                                            # codebook=vq_model.quantizer.codebooks[0] if opt.fix_token_emb else None,
                                            share_weight=res_opt.share_weight,
                                            clip_version=clip_version,
                                            opt=res_opt)

    ckpt = torch.load(pjoin(res_opt.checkpoints_dir, res_opt.dataset_name, res_opt.name, 'model', 'net_best_fid.tar'),
                      map_location=opt.device)
    missing_keys, unexpected_keys = res_transformer.load_state_dict(ckpt['res_transformer'], strict=False)
    assert len(unexpected_keys) == 0
    assert all([k.startswith('clip_model.') for k in missing_keys])
    print(f'Loading Residual Transformer {res_opt.name} from epoch {ckpt["ep"]}!')
    return res_transformer

In [90]:
res_opt_path = pjoin('checkpoints', opt.dataset_name, opt.res_name, 'opt.txt')
res_opt = get_opt(res_opt_path, device=opt.device)
res_model = load_res_model(res_opt,vq_opt)

Reading checkpoints/t2m/tres_nlayer8_ld384_ff1024_rvq6ns_cdp0.2_sw/opt.txt
latent_dim: 384, ff_size: 1024, nlayers: 8, nheads: 6, dropout: 0.2


Loading CLIP...
Loading Residual Transformer tres_nlayer8_ld384_ff1024_rvq6ns_cdp0.2_sw from epoch 440!


Joints to features code here 

In [91]:
import torch
import numpy as np
from common.skeleton import Skeleton
from utils.paramUtil import *
from common.quaternion import qrot_np, qinv_np, qmul_np, quaternion_to_cont6d_np, qbetween_np, qfix

def joints_to_features(joints, feet_thre=0.002, dataset='humanml'):
    """
    joints: numpy array of shape (frames, joints_num, 3)
    returns: RIC feature vector (frames-1, feature_dim)
    """

    if dataset == 'humanml':
        n_raw_offsets = torch.from_numpy(t2m_raw_offsets)
        kinematic_chain = t2m_kinematic_chain
        joints_num = 22
        fid_r, fid_l = [8, 11], [7, 10]
        face_joint_indx = [2, 1, 17, 16]
    elif dataset == 'kit':
        n_raw_offsets = torch.from_numpy(kit_raw_offsets)
        kinematic_chain = kit_kinematic_chain
        joints_num = 21
        fid_r, fid_l = [14, 15], [19, 20]
        face_joint_indx = [11, 16, 5, 8]
    else:
        raise ValueError('Unknown dataset')

    joints = torch.from_numpy(joints).float()

    # Step 1: Build Skeleton
    skel = Skeleton(n_raw_offsets, kinematic_chain, 'cpu')

    # Step 2: Uniform skeleton (optional, skipped if already processed)
    tgt_offsets = skel.get_offsets_joints(joints[0])

    # Step 3: Put on floor
    floor_height = joints[:, :, 1].min()
    joints[:, :, 1] -= floor_height

    # Step 4: Face Z+
    r_hip, l_hip, sdr_r, sdr_l = face_joint_indx
    across1 = joints[0, r_hip] - joints[0, l_hip]
    across2 = joints[0, sdr_r] - joints[0, sdr_l]
    across = across1 + across2
    across = across / torch.norm(across)
    forward_init = torch.cross(torch.tensor([0.0, 1.0, 0.0]), across)
    forward_init = forward_init / torch.norm(forward_init)
    target = torch.tensor([0.0, 0.0, 1.0])
    root_quat_init = qbetween_np(forward_init.numpy(), target.numpy())
    root_quat_init = np.ones(joints.shape[:-1] + (4,)) * root_quat_init
    joints = qrot_np(root_quat_init, joints.numpy())

    global_positions = joints.copy()

    # Step 5: Foot contact detection
    def foot_detect(positions, thres):
        velfactor = np.array([thres, thres])
        feet_l_x = (positions[1:, fid_l, 0] - positions[:-1, fid_l, 0]) ** 2
        feet_l_y = (positions[1:, fid_l, 1] - positions[:-1, fid_l, 1]) ** 2
        feet_l_z = (positions[1:, fid_l, 2] - positions[:-1, fid_l, 2]) ** 2
        feet_l = ((feet_l_x + feet_l_y + feet_l_z) < velfactor).astype(np.float32)

        feet_r_x = (positions[1:, fid_r, 0] - positions[:-1, fid_r, 0]) ** 2
        feet_r_y = (positions[1:, fid_r, 1] - positions[:-1, fid_r, 1]) ** 2
        feet_r_z = (positions[1:, fid_r, 2] - positions[:-1, fid_r, 2]) ** 2
        feet_r = ((feet_r_x + feet_r_y + feet_r_z) < velfactor).astype(np.float32)

        return feet_l, feet_r

    feet_l, feet_r = foot_detect(joints, feet_thre)

    # Step 6: Extract features

    # Inverse kinematics to quaternions
    skel = Skeleton(n_raw_offsets, kinematic_chain, 'cpu')
    quat_params = skel.inverse_kinematics_np(joints, face_joint_indx, smooth_forward=True)
    quat_params = qfix(quat_params)

    # Continuous 6D
    cont_6d_params = quaternion_to_cont6d_np(quat_params)
    r_rot = quat_params[:, 0]

    # Root linear velocities
    velocity = joints[1:, 0] - joints[:-1, 0]
    velocity = qrot_np(r_rot[1:], velocity)
    l_velocity = velocity[:, [0, 2]]

    # Root rotation velocities
    r_velocity = qmul_np(r_rot[1:], qinv_np(r_rot[:-1]))
    r_velocity = np.arcsin(r_velocity[:, 2:3])

    # Root height
    root_y = joints[:-1, 0, 1:2]

    root_data = np.concatenate([r_velocity, l_velocity, root_y], axis=-1)

    # Joint positions (RIC)
    joints_local = joints.copy()
    joints_local[..., 0] -= joints_local[:, 0:1, 0]
    joints_local[..., 2] -= joints_local[:, 0:1, 2]
    joints_local = qrot_np(np.repeat(r_rot[:, None], joints_local.shape[1], axis=1), joints_local)
    ric_data = joints_local[:, 1:].reshape(joints_local.shape[0], -1)

    # Joint rotations
    rot_data = cont_6d_params[:, 1:].reshape(cont_6d_params.shape[0], -1)

    # Joint velocities
    local_vel = qrot_np(np.repeat(r_rot[:-1, None], global_positions.shape[1], axis=1),
                        global_positions[1:] - global_positions[:-1])
    local_vel = local_vel.reshape(local_vel.shape[0], -1)

    # Combine
    data = np.concatenate([root_data, ric_data[:-1], rot_data[:-1], local_vel, feet_l, feet_r], axis=-1)

    return data


In [92]:
test = np.load(r"generation/generation_name_nopredlen/joints/0/sample0_repeat2_len196_ik.npy", allow_pickle=True)  

In [93]:
ric_features = joints_to_features(test, feet_thre=0.002, dataset='humanml')
print("RIC Features Shape:", ric_features.shape)


RIC Features Shape: (195, 263)


In [94]:
mean = np.load(pjoin('checkpoints', opt.dataset_name, model_opt.vq_name, 'meta', 'mean.npy'))
std = np.load(pjoin('checkpoints', opt.dataset_name, model_opt.vq_name, 'meta', 'std.npy'))

In [95]:
normed_joints = (ric_features - mean) / std

In [96]:
import torch

ric_tensor = torch.from_numpy(normed_joints).float().unsqueeze(0).to('cpu')  # (1, seq_len, 263)



In [97]:
quantized, tokens = vq_model.encode(ric_tensor)

Inbetween code

In [98]:
FPS = 20
t_sec=10 # addtion at this second
downsample_factor=5
frame_index = int(t_sec * FPS)
insert_frame = int(frame_index / downsample_factor)

In [99]:
prompt_list = [
    "the person runs in forward direction",
     
]
est_length = True
length_list = []

# if est_length:
#     print("Since no motion length are specified, we will use estimated motion lengthes!!")
#     text_embedding = t2m_transformer.encode_text(prompt_list)
#     pred_dis = length_estimator(text_embedding)
#     probs = F.softmax(pred_dis, dim=-1)  # (b, ntoken)
#     token_lens = Categorical(probs).sample()  # (b, seqlen)
#     # lengths = torch.multinomial()
# else:
token_lens = torch.LongTensor(length_list) // 4
token_lens = token_lens.to(opt.device).long()

m_length = token_lens * 4
captions = prompt_list

sample = 0
kinematic_chain = t2m_kinematic_chain
converter = Joint2BVHConvertor()

In [100]:
t2m_transformer = load_trans_model(model_opt , opt ,'latest.tar')

latent_dim: 384, ff_size: 1024, nlayers: 8, nheads: 6, dropout: 0.2


Loading CLIP...
Loading Transformer 2024-02-14-14-27-29_8_GPT_officialTrans_2iterPrdictEnd from epoch 1999!


In [101]:
prompt_list = [
    "the person runs in forward direction",
     
]
est_length = True
length_list = []

# if est_length:
#     print("Since no motion length are specified, we will use estimated motion lengthes!!")
#     text_embedding = t2m_transformer.encode_text(prompt_list)
#     pred_dis = length_estimator(text_embedding)
#     probs = F.softmax(pred_dis, dim=-1)  # (b, ntoken)
#     token_lens = Categorical(probs).sample()  # (b, seqlen)
#     lengths = torch.multinomial()
# else:
token_lens = torch.LongTensor(length_list) // 4
token_lens = token_lens.to(opt.device).long()

m_length = token_lens * 4
captions = prompt_list

sample = 0
kinematic_chain = t2m_kinematic_chain
converter = Joint2BVHConvertor()

In [103]:
torch.set_default_dtype(torch.float32)
t2m_transformer = t2m_transformer.float()
res_model = res_model.float()
vq_model = vq_model.float()

In [106]:
def inv_transform(data):
       return data * std + mean

In [ ]:
with torch.no_grad():
    mids, pred_len = t2m_transformer.generate(captions, token_lens,
                                    timesteps=opt.time_steps,
                                    cond_scale=opt.cond_scale,
                                    temperature=opt.temperature,
                                    topk_filter_thres=opt.topkr,
                                    gsample=opt.gumbel_sample,
                                    is_predict_len=opt.motion_length==-1
                                    )
    token_lens = pred_len
    m_length = token_lens*4
    # print(mids)
    # print(mids.shape)
    mids_1 = res_model.generate(mids, captions, token_lens, temperature=1, cond_scale=5)
    pred_motions = vq_model.forward_decoder(mids_1)
    pred_motions_1 = pred_motions.detach().numpy()
             
    

In [ ]:


# Extract parts from quantized
quantized_base = quantized[0, :]  # Shape: [L, 6]
part1 = quantized_base[:insert_frame]        # Quantized before 10s
part2 = quantized_base[insert_frame:]        # Quantized after 10s


motion_A = part1[:, 0]        # Last tokens before 10s
motion_B = mids_1[0, : , 0]     # New motion to insert
motion_C = part2[:, 0]        # Tokens after 10s

# Calculate actual lengths
len_A = len(motion_A)
len_B = len(motion_B)
len_C = len(motion_C)

# Adjust half_token_length based on minimum available length
half_token_length = min(24, len_A, len_B, len_C)
num_transition_token = 2

# Create tokens tensor with appropriate size
tokens = -1 * torch.ones((2, half_token_length*2 + num_transition_token), dtype=torch.long, device=opt.device)

# === Transition 1: part1 (A) to mids (B) ===
tokens[0, :half_token_length] = motion_A[-half_token_length:]
tokens[0, half_token_length:half_token_length + num_transition_token] = t2m_transformer.pad_id
tokens[0, half_token_length + num_transition_token:] = motion_B[:half_token_length]

# === Transition 2: mids (B) to part2 (C) ===
tokens[1, :half_token_length] = motion_B[-half_token_length:]
tokens[1, half_token_length:half_token_length + num_transition_token] = t2m_transformer.pad_id
tokens[1, half_token_length + num_transition_token:] = motion_C[:half_token_length]

# Debug prints
print(f"Lengths - A: {len_A}, B: {len_B}, C: {len_C}")
print(f"Using half_token_length: {half_token_length}")
print(f"Token shape: {tokens.shape}")
z
# Predict transitions
tokens[tokens == -1] = t2m_transformer.pad_id
inpainting_mask = tokens == t2m_transformer.pad_id
inpaint_index = t2m_transformer.edit2(None, tokens)

# Extract transitions
transition1 = inpaint_index[0, inpainting_mask[0]].unsqueeze(-1)
transition1 = torch.nn.functional.pad(transition1, (0, 5), value=-1)

transition2 = inpaint_index[1, inpainting_mask[1]].unsqueeze(-1)
transition2 = torch.nn.functional.pad(transition2, (0, 5), value=-1)

# Construct final motion
final_motion = torch.cat([
    part1,         # original motion up to 10s
    transition1,   # transition: original → mids
    mids_1[0][:pred_len[0]],       # inserted new motion
    transition2,   # transition: mids → original continuation
    part2          # rest of original motion
], dim=0).unsqueeze(0)  # Add batch dim

# Final result
quantized = final_motion


Lengths - A: 40, B: 50, C: 8
Using half_token_length: 8
Token shape: torch.Size([2, 18])


In [110]:
pred_motions = vq_model.forward_decoder(mids_1)

In [111]:
data = inv_transform(pred_motions.detach().numpy())

In [117]:
import threading
combine = True  # Set to True if you want to combine the data, False otherwise
def export_bvh(k, joint_data):
            length = m_length[k]
            animation_path = pjoin(animation_dir, str(k))
            joint_path = pjoin(joints_dir, str(k))
            os.makedirs(animation_path, exist_ok=True)
            os.makedirs(joint_path, exist_ok=True)
            joint_data = joint_data if combine else joint_data[:length]
            joint = recover_from_ric(torch.from_numpy(joint_data).float().pin_memory().to(opt.device), 22).cpu().numpy()
            bvh_path = pjoin(animation_path, f"sample{k}_inmiddle_{length}_ik.bvh")
            converter.convert(joint, filename=bvh_path, iterations=50)
            bvh_path = pjoin(animation_path, f"sample{k}_inmiddle_len{length}.bvh")
            converter.convert(joint, filename=bvh_path, iterations=50, foot_ik=False)

threads = [threading.Thread(target=export_bvh, args=(k, joint_data)) for k, joint_data in enumerate(data)]
for t in threads:
    t.start()
for t in threads:
    t.join()
        


do all the other processing same